In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
import experiment_manager as em
import far_ho as far

#### Checking implementation of `ForwardHG`. The hypergradients computed with `ForwardHG` must be the same as the one computed by `ReverseHG`.

In [ ]:
def _test(method):
    try: ss.close()
    except: pass
    tf.reset_default_graph()
    ss = tf.InteractiveSession()
    v1 = tf.Variable([[1., -1., 1.], [2., 10., 2.]])

    v2 = tf.Variable([-1., 1., 0.2])
    lmbd = far.get_hyperparameter('lambda', 
                              initializer=tf.ones_initializer,
                             shape=v2.get_shape(), scalar=True)

    cost = tf.reduce_mean(v1**2) + tf.reduce_sum(lmbd*v2**2)
#     cost = tf.reduce_sum(lmbd*v2**2)

    
    
    _et = far.get_hyperparameter('eta', 0.1)
    io_optim = far.MomentumOptimizer(_et, far.get_hyperparameter('mu', .5))
    # io_optim = far.GradientDescentOptimizer(_et)

    io_optim_dict = io_optim.minimize(cost) 
    oo = tf.reduce_mean(v1*v2)
#     oo = tf.reduce_mean(v2)
    optim_oo = tf.train.AdamOptimizer(.01)
    farho = far.HyperOptimizer(hypergradient=method())
    farho.minimize(oo, optim_oo, cost, io_optim)
    return ss, farho, cost, oo

#### FORWARD

In [ ]:
ss, farho, cost, oo = _test(far.ForwardHG)

tf.global_variables_initializer().run()

# execution with gradient descent! (looks ol)
farho.run(10, _skip_hyper_ts=True)

hypergrads_fhg = ss.run(far.utils.hypergradients())
hypergrads_fhg

#### REVERSE

In [ ]:
ss, farho, cost, oo = _test(far.ReverseHg)

tf.global_variables_initializer().run()

# execution with gradient descent! (looks ol)
farho.run(10, _skip_hyper_ts=True)

hypergrads_rev = ss.run(far.utils.hypergradients())
hypergrads_rev

In [ ]:
print(np.linalg.norm(np.array(hypergrads_fhg) - np.array(hypergrads_rev)))

#### OK... SOUNDS GOOD